We will show the main RePlay functionality and compare performance of RePlay models on well-known MovieLens dataset. For simplicity we consider here only the variou bandit algorithms with context. The list of considered strategies for comparison:

1) Linear UCB

2) Linear Thompson Sampling

3) Logistic Thompson Sampling

### Dataset

We will compare RePlay models on MovieLens 1m.

### Dataset preprocessing:

Ratings greater than or equal to 3 are considered as positive interactions.

### Data split

Dataset is split by date so that 20% of the last interactions as are placed in the test part. Cold items and users are dropped.

### Predict:
We will predict top-10 most relevant films for each user.

### Metrics

Quality metrics used: ndcg@k, hitrate@k, map@k, mrr@k for k = 1, 5, 10 Additional metrics used: coverage@k and surprisal@k.


In [1]:
# ! pip install rs-datasets

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%config Completer.use_jedi = False

In [4]:
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)

In [5]:
import logging
import time
import pandas as pd
import numpy as np

from pyspark.sql import functions as sf
from pyspark.sql import types as st
# from pyspark.sql.types import IntegerType

from replay.data import Dataset, FeatureHint, FeatureInfo, FeatureSchema, FeatureType
from replay.data.dataset_utils import DatasetLabelEncoder
from replay.metrics import Coverage, HitRate, MRR, MAP, NDCG, Surprisal, Experiment, OfflineMetrics

# from replay.experimental.preprocessing.data_preparator import DataPreparator, Indexer
from replay.models import (
    UCB,
    Wilson, 
    Word2VecRec,
    RandomRec,
    LinUCB, #added LinUCB (disjoint version)
)

from replay.models.base_rec import HybridRecommender
from replay.utils.session_handler import State
from replay.splitters import TimeSplitter
from replay.utils.spark_utils import convert2spark, get_log_info
from rs_datasets import MovieLens

In [6]:
spark = State().session
spark

24/08/23 13:34:28 WARN Utils: Your hostname, sudakovcom-MS-7D48 resolves to a loopback address: 127.0.1.1; using 10.255.173.26 instead (on interface enp3s0)
24/08/23 13:34:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/23 13:34:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/23 13:34:28 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [7]:
spark.sparkContext.setLogLevel('ERROR')

In [8]:
logger = logging.getLogger("replay")

In [9]:
K = 10
K_list_metrics = [1, 5, 10]
BUDGET = 20
BUDGET_NN = 10
SEED = 12345

## Preprocessing 

### Data loading

In [10]:
data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [11]:
interactions_spark = convert2spark(data.ratings)
interactions_spark.show(5)

/home/sudakovcom/Desktop/RePlayHDILab2024/.conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:471: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      1|   1193|     5|978300760|
|      1|    661|     3|978302109|
|      1|    914|     3|978301968|
|      1|   3408|     4|978300275|
|      1|   2355|     5|978824291|
+-------+-------+------+---------+
only showing top 5 rows



In [12]:
# will consider ratings >= 3 as positive feedback. A positive feedback is treated with rating = 1
only_positives_interactions = interactions_spark.filter(sf.col('rating') >= 3).withColumn('rating', sf.lit(1))
only_positives_interactions.count()

836478

In [13]:
only_positives_interactions.show(2)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      1|   1193|     1|978300760|
|      1|    661|     1|978302109|
+-------+-------+------+---------+
only showing top 2 rows



### Data split

In [14]:
# train/test split 
train_spl = TimeSplitter(
    time_threshold=0.2,
    drop_cold_items=True,
    drop_cold_users=True,
    query_column="user_id",
)
train, test = train_spl.split(only_positives_interactions)
print('train info:\n', get_log_info(train, user_col="user_id", item_col="item_id"))
print('test info:\n', get_log_info(test, user_col="user_id", item_col="item_id"))

train info:
 total lines: 669181, total users: 5397, total items: 3569
test info:
 total lines: 86542, total users: 1139, total items: 3279


In [15]:
train.is_cached

False

In [16]:
# train/test split for hyperparameters selection
opt_train, opt_val = train_spl.split(train)
opt_train.count(), opt_val.count()

(535343, 24241)

In [17]:
opt_train.is_cached

False

In [18]:
# negative feedback will be used for Wilson and UCB models
only_negatives_log = interactions_spark.filter(sf.col('rating') < 3).withColumn('rating', sf.lit(0.))
test_start = test.agg(sf.min('timestamp')).collect()[0][0]

# train with both positive and negative feedback
pos_neg_train=(train
              .withColumn('rating', sf.lit(1.))
              .union(only_negatives_log.filter(sf.col('timestamp') < test_start))
             )
pos_neg_train.cache()
pos_neg_train.count()

798993

In [19]:
pos_neg_train.is_cached

True

### Item features 

In [20]:
item_features = convert2spark(data.items)
# item_features  = item_features.withColumnRenamed('item_id', 'item_idx')
item_features.show(5)

+-------+--------------------+--------------------+
|item_id|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|        Comedy|Drama|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [21]:
item_features.select(sf.min('item_id')).show()
item_features.select(sf.max('item_id')).show()
#just to check that the indexing is dense between 0 and 3882
item_features.count()

+------------+
|min(item_id)|
+------------+
|           1|
+------------+

+------------+
|max(item_id)|
+------------+
|        3952|
+------------+



3883

In [22]:
year = item_features.withColumn('year', sf.substring(sf.col('title'), -5, 4).astype(st.IntegerType())).select('item_id', 'year')
year.show(2)

+-------+----+
|item_id|year|
+-------+----+
|      1|1995|
|      2|1995|
+-------+----+
only showing top 2 rows



In [23]:
genres = (
    item_features.select(
        sf.col("item_id"),
        sf.split("genres", "\|").alias("genres")
    )
)

In [24]:
genres.show()

+-------+--------------------+
|item_id|              genres|
+-------+--------------------+
|      1|[Animation, Child...|
|      2|[Adventure, Child...|
|      3|   [Comedy, Romance]|
|      4|     [Comedy, Drama]|
|      5|            [Comedy]|
|      6|[Action, Crime, T...|
|      7|   [Comedy, Romance]|
|      8|[Adventure, Child...|
|      9|            [Action]|
|     10|[Action, Adventur...|
|     11|[Comedy, Drama, R...|
|     12|    [Comedy, Horror]|
|     13|[Animation, Child...|
|     14|             [Drama]|
|     15|[Action, Adventur...|
|     16|   [Drama, Thriller]|
|     17|    [Drama, Romance]|
|     18|          [Thriller]|
|     19|            [Comedy]|
|     20|            [Action]|
+-------+--------------------+
only showing top 20 rows



In [25]:
genres_list = (
    genres.select(sf.explode("genres").alias("genre"))
    .distinct().filter('genre <> "(no genres listed)"')
    .toPandas()["genre"].tolist()
)

In [26]:
genres_list

['Mystery',
 'Action',
 'Documentary',
 "Children's",
 'Drama',
 'Adventure',
 'Film-Noir',
 'Crime',
 'Animation',
 'Fantasy',
 'Comedy',
 'Western',
 'Romance',
 'Thriller',
 'War',
 'Sci-Fi',
 'Musical',
 'Horror']

In [27]:
item_features = genres
for genre in genres_list:
    item_features = item_features.withColumn(
        genre,
        sf.array_contains(sf.col("genres"), genre).astype(st.IntegerType())
    )
item_features = item_features.drop("genres").cache()
item_features.count()
item_features = item_features.join(year, on='item_id', how='inner')
item_features.cache()

DataFrame[item_id: int, Mystery: int, Action: int, Documentary: int, Children's: int, Drama: int, Adventure: int, Film-Noir: int, Crime: int, Animation: int, Fantasy: int, Comedy: int, Western: int, Romance: int, Thriller: int, War: int, Sci-Fi: int, Musical: int, Horror: int, year: int]

In [28]:
item_features = item_features.withColumnRenamed("Children's","Children")
item_features.cache()

DataFrame[item_id: int, Mystery: int, Action: int, Documentary: int, Children: int, Drama: int, Adventure: int, Film-Noir: int, Crime: int, Animation: int, Fantasy: int, Comedy: int, Western: int, Romance: int, Thriller: int, War: int, Sci-Fi: int, Musical: int, Horror: int, year: int]

### Users features

In [29]:
data.users.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [30]:
#same preprocessing for users as was done in 2.4.1.
user_features = convert2spark(data.users)
#switch for a while into pandas
user_features = user_features.toPandas()
user_features.head(2)

/home/sudakovcom/Desktop/RePlayHDILab2024/.conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:471: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072


In [31]:
print("max ocupation index: ", user_features['occupation'].max())
print("min ocupation index: ", user_features['occupation'].min())
count_diff_zips = user_features['zip_code'].unique().size
print("different zip codes: ", count_diff_zips) #ok, too much different zip codes, let us drop them for now
users_pd = user_features.drop(columns=['zip_code'])
users_pd.head()
#binarize age variable
bins = [0, 20, 30, 40, 50, 60, np.inf]
names = ['<20', '20-29', '30-39','40-49', '51-60', '60+']

users_pd['agegroup'] = pd.cut(users_pd['age'], bins, labels=names)

#binarize following https://github.com/kfoofw/bandit_simulations/tree/master
cat_embed_cols = ["agegroup","gender","occupation", "year"]
continuous_cols = ["age", "Crime", "Sci-Fi", "Musical", 
                   "Mystery", "Documentary", "Fantasy", "Children", 
                   "Drama", "Horror", "Adventure", "Western","Romance", 
                   "War", "Animation", "Action", "Comedy", "Thriller",
                   "Film-Noir"]
wide_cols = ["agegroup"]

max ocupation index:  20
min ocupation index:  0
different zip codes:  3439


In [32]:
users_pd.head()

,user_id,gender,age,occupation,agegroup
0,1,F,1,10,<20
1,2,M,56,16,51-60
2,3,M,25,15,20-29
3,4,M,45,7,40-49
4,5,M,25,20,20-29


In [33]:
#make it pyspark
user_features = spark.createDataFrame(users_pd) 
user_features.printSchema()
print("total users: ",user_features.count())

root
 |-- user_id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- occupation: integer (nullable = true)
 |-- agegroup: string (nullable = true)

total users:  6040


/home/sudakovcom/Desktop/RePlayHDILab2024/.conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:471: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


In [34]:
type(user_features)

pyspark.sql.dataframe.DataFrame

In [35]:
feature_schema = FeatureSchema(
    [
        FeatureInfo(
            column="user_id",
            feature_type=FeatureType.CATEGORICAL,
            feature_hint=FeatureHint.QUERY_ID,
        ),
        FeatureInfo(
            column="item_id",
            feature_type=FeatureType.CATEGORICAL,
            feature_hint=FeatureHint.ITEM_ID,
        ),
        FeatureInfo(
            column="rating",
            feature_type=FeatureType.NUMERICAL,
            feature_hint=FeatureHint.RATING,
        ),
        FeatureInfo(
            column="timestamp",
            feature_type=FeatureType.NUMERICAL,
            feature_hint=FeatureHint.TIMESTAMP,
        ),
    ]
)

In [36]:
all_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=interactions_spark,
    item_features=item_features,
    query_features=user_features,
)

train_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=train,
    item_features=item_features,
    query_features=user_features,
)

test_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=test,
    item_features=item_features,
    query_features=user_features,
)

train_neg_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=pos_neg_train,
    item_features=item_features,
    query_features=user_features,
)

opt_train_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=opt_train,
    item_features=item_features,
    query_features=user_features,
)

opt_val_dataset = Dataset(
    feature_schema=feature_schema,
    interactions=opt_val,
    item_features=item_features,
    query_features=user_features,
)

In [37]:
encoder = DatasetLabelEncoder()
encoder.fit(all_dataset)
train_dataset = encoder.transform(train_dataset)
train_neg_dataset = encoder.transform(train_neg_dataset)
test_dataset = encoder.transform(test_dataset)
opt_train_dataset = encoder.transform(opt_train_dataset)
opt_val_dataset = encoder.transform(opt_val_dataset)

In [38]:
opt_train_dataset.interactions.show(2)

+------+---------+-------+-------+
|rating|timestamp|user_id|item_id|
+------+---------+-------+-------+
|     1|974728976|   2441|     93|
|     1|974728976|   2441|    425|
+------+---------+-------+-------+
only showing top 2 rows



In [39]:
e = Experiment(
    [
        MAP(K),
        NDCG(K),
        HitRate(K_list_metrics),
        Coverage(K),
        Surprisal(K),
        MRR(K)
    ],
    test_dataset.interactions,
    train_dataset.interactions,
    query_column=train_dataset.feature_schema.query_id_column,
    item_column=train_dataset.feature_schema.item_id_column,
    rating_column=train_dataset.feature_schema.interactions_rating_column,
    )

In [40]:
bandit_models = {
    # 'Random (uniform)': [RandomRec(seed=SEED, distribution='uniform'), 'no_opt'],
    # 'Random (popularity-based)': [RandomRec(seed=SEED, distribution='popular_based'), {"alpha": [-0.5, 100]}],
    # 'UCB': [UCB(exploration_coef=2.0), 'no_opt'], #2.0 as default, 0.5 as original 
    # 'Wilson': [Wilson(), 'no_opt'],
    'Linear UCB (disjoint models)': [LinUCB(eps = 0.0, alpha = 1.0, regr_type = 'disjoint'), 'no_opt'],
}

In [41]:
for name, [model, params] in bandit_models.items():
    print(name)
    print(model)
    print(params)

Linear UCB (disjoint models)
LinUCB
no_opt


In [42]:
bandit_models.items()

dict_items([('Linear UCB (disjoint models)', [<replay.models.lin_ucb.LinUCB object at 0x7aead4572640>, 'no_opt'])])

In [43]:
def fit_predict_add_res(name, model, experiment, train, test, suffix=''):
    """
    Run fit_predict for the `model`, measure time on fit_predict and evaluate metrics
    """
    start_time=time.time()
    
    dataset = {'dataset': train_dataset}
    predict_params = {'k': K, 'queries': test_dataset.query_ids}
    
    if isinstance(model, (Wilson, UCB, LinUCB)):
        dataset['dataset'] = train_neg_dataset

    predict_params.update(dataset)

    model.fit(**dataset)
    fit_time = time.time() - start_time

    pred=model.predict(**predict_params)
    pred.cache()
    pred.count()
    predict_time = time.time() - start_time - fit_time

    experiment.add_result(name + suffix, pred)
    metric_time = time.time() - start_time - fit_time - predict_time
    experiment.results.loc[name + suffix, 'fit_time'] = fit_time
    experiment.results.loc[name + suffix, 'predict_time'] = predict_time
    experiment.results.loc[name + suffix, 'metric_time'] = metric_time
    experiment.results.loc[name + suffix, 'full_time'] = (fit_time + 
                                                          predict_time +
                                                          metric_time)
    pred.unpersist()
    print(experiment.results[['NDCG@{}'.format(K), 'MRR@{}'.format(K), 'Coverage@{}'.format(K), 'fit_time']].sort_values('NDCG@{}'.format(K), ascending=False))

In [44]:
def full_pipeline(models, experiment, train, test, suffix='', budget=BUDGET):
    """
    For each model:
        -  if required: run hyperparameters search, set best params and save param values to `experiment`
        - pass model to `fit_predict_add_res`        
    """
    
    for name, [model, params] in models.items():
        model.logger.info(msg='{} started'.format(name))
        if params != 'no_opt':
            model.logger.info(msg='{} optimization started'.format(name))
            best_params = model.optimize(opt_train, 
                                         opt_val, 
                                         param_borders=params, 
                                         item_features=item_features,
                                         user_features=user_features,
                                         k=K, 
                                         budget=budget)
            logger.info(msg='best params for {} are: {}'.format(name, best_params))
            model.set_params(**best_params)
        
        logger.info(msg='{} fit_predict started'.format(name))
        fit_predict_add_res(name, model, experiment, train, test, suffix)
        # here we call protected attribute to get all parameters set during model initialization
        experiment.results.loc[name + suffix, 'params'] = str(model._init_args)

In [45]:
%%time
full_pipeline(bandit_models, e, train, test)

23-Aug-24 13:34:52, replay, INFO: Linear UCB (disjoint models) started
23-Aug-24 13:34:52, replay, INFO: Linear UCB (disjoint models) fit_predict started


TypeError: can't multiply sequence by non-int of type 'str'

In [46]:
e.results.sort_values('NDCG@10', ascending=False)

KeyError: 'NDCG@10'